In [ ]:
# Obtengo la ultima version de TensorFlow
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
# Importo Librerías
from tensorflow.keras.applications.nasnet import NASNetMobile 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model  
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
import math
import os

In [ ]:
# Conecto con Google Drive para obtener el dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("drive/My Drive/Colab Notebooks/Cat-Dog Classifier")
os.listdir()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['train_catdog.zip', 'Cat Dog Classifier DNN.ipynb']

In [ ]:
# Copio el dataset a Google Colab desde Google Drive, y lo deszipeo en una carpeta nueva
# Este método es mucho más performante que leer los archivos directamente desde Google Drive
!cp "train_catdog.zip"
!mkdir "train_catdog"
!unzip -q train_catdog.zip -d "train_catdog"
!ls

cp: missing destination file operand after 'train_catdog.zip'
Try 'cp --help' for more information.
'Cat Dog Classifier DNN.ipynb'	 train_catdog   train_catdog.zip


In [ ]:
# Utilizo como CNN base el modelo NASNetMobile
pre_trained_model = NASNetMobile(input_shape=(224,224,3), 
                                include_top = False,
                                weights = 'imagenet')

for layer in pre_trained_model.layers:    
    layer.trainable = False     

# Cargamos nuestras capas de clasificación propias
x = layers.Flatten() (pre_trained_model.output)
x = layers.Dropout(0.5) (x)
x = layers.Dense(256, activation='relu') (x)
x = layers.Dense(1, activation='sigmoid') (x)

# Consolidamos el modelo y seteo la funcion de pérdida (los function) y qué optimizador para reducirla
model = Model(pre_trained_model.input, x)
model.compile (optimizer= RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
# model.summary()

19996672/19993432 [==============================] - 0s 0us/step


In [ ]:
# Normalizo los valores de los pixeles entre 0 y 1
# Uso Data Augmentation agregando imagenes
# Reservo 20% de imagenes para el validation set
train_dir = os.path.join("train_catdog")
train_datagen = ImageDataGenerator(
                                rescale = 1./255., 
                                horizontal_flip = True, 
                                rotation_range=20,                         
                                zoom_range=0.1,
                                width_shift_range=0.1,
                                height_shift_range=0.1,                                
                                validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
                                train_dir, 
                                batch_size = 64, 
                                class_mode = 'binary', 
                                target_size = (224, 224), 
                                color_mode = 'rgb',
                                interpolation = 'nearest',
                                subset='training')
validation_generator = train_datagen.flow_from_directory(
                                train_dir, 
                                batch_size = 64, 
                                class_mode = 'binary', 
                                target_size = (224, 224),  
                                color_mode = 'rgb', 
                                interpolation = 'nearest',
                                subset='validation')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Agrego Early Stopping para dejar de entrenar el modelo si comienza a haber overfitting
earlystop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=9)

# Agrego Model Checkpoint para almacenar el modelo más preciso a medida que avanza el entrenamiento
modelcheckpoint_callback = tf.keras.callbacks.ModelCheckpoint(  
                                  filepath = 'CatDog_Classifier_Model',
                                  save_weights_only=False,
                                  monitor='val_acc',
                                  mode='max',
                                  save_best_only=True,
                                  save_freq='epoch',
                                  verbose=1)

# Agrego Learning Rate Scheduler, cada 3 epochs divido el learning rate por la mitad
def step_decay(epoch):
  initial_lrate = 0.01
  drop = 0.5
  epochs_drop = 3.0
  lrate = initial_lrate * math.pow(drop,  
          math.floor((1+epoch)/epochs_drop))
  return lrate

lrscheduler_callback = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            # steps_per_epoch = XX, # Si no se envia se calcula por defecto. Cantidad de muestras training / training batch size
            epochs = 100,  
            # validation_steps = XX, # Si no se envia se calcula por defecto. Cantidad de muestras validation / validation batch size
            callbacks=[earlystop_callback, modelcheckpoint_callback, lrscheduler_callback],
            verbose = 1) 


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/100
313/313 [==============================] - ETA: 0s - loss: 2.5036 - acc: 0.9664
Epoch 00001: val_acc improved from -inf to 0.97360, saving model to CatDog_Classifier_Model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: CatDog_Classifier_Model/assets
313/313 [==============================] - 2855s 9s/step - loss: 2.5036 - acc: 0.9664 - val_loss: 0.0779 - val_acc: 0.9736 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01.
Epoch 2/100
313/313 [==============================] - ETA: 0s - loss: 0.2638 - acc: 0.9714
Epoch 00002: val_acc improved from 0.97360 to 0.97560, saving model to CatDog_Classifier_Model
INFO:tensorflow:Assets written to: CatDog_Classifier_Model/assets
313/313 [==============================] - 2818s 9s/step - loss: 0.2638 - acc: 0.9714 - val_loss: 0.1848 - val_acc: 0.9756 - lr: 0.0100

Epoch 0